In [ ]:
# import findspark
# findspark.init()
# import pyspark
# # import random
# sc = pyspark.SparkContext(appName="Pi")
# # num_samples = 1000
# # def inside(p):     
# #   x, y = random.random(), random.random()
# #   return x*x + y*y < 1
# # count = sc.parallelize(range(0, num_samples)).filter(inside).count()
# # pi = 4 * count / num_samples
# # print(pi)
# print(sc.version)
# print(sc.version)
# print(sc.pythonVer)
# print(sc.localr)
# sc.stop()

# SparkContext

# RDD's

# Pair RDD's

# PySpark SQL, DataFrames, and SparkSession
### PySparkSQL works on DataFrames not RDD's

# Spark Wrangling, Feature Generation

In [ ]:
# Dropping Values
df = df.dropna()  # Drop any records with NULL values

# drop records if both LISTPRICE and SALESCLOSEPRICE are NULL
# any = if any values are null, all = only if all values are null
df = df.dropna(how='all', subset['LISTPRICE', 'SALESCLOSEPRICE '])
df = df.dropna(thresh=2) # Drop records where at least two columns have NULL values
df.dropDuplicates()
df.dropDuplicates(['streetaddress']) # Check only a column list

cols_to_drop = ['STREETNUMBERNUMERIC', 'LOTSIZEDIMENSIONS']
df = df.drop(*cols_to_drop) # Drop columns from list using '*'

# Text Filtering
df = df.where(~df['column'].like('some text or value')) # ~ the tilde is the NOT condition 

# Scaling
# (original value - min) / (max - min)

# Standardization
# (original value - mean) / (stddev)

# Log Scaling
from pyspark.sql.functions import log
df.withColumn('new_column', log(df['old_column']))

# Imputation
df.fillna(valueToFillWith, list_of_columns)

# PySpark DataFrame Join
df.join(other_df, on='column with keys to join', how='left')

# Date objects
from pyspark.sql.functions import to_date, year, month, day, dayofmonth, weekofyear, datediff
df = df.withColumn('new_col', to_date('col2'))
df = df.withColumn('new_col', dayofmonth('col2'))
df = df.withColumn('new_col', datediff('col2', 'col3'))

# Lagging Features
from pyspark.sql.functions import to_date, lag
from pyspark.sql.window import Window

w = Window().orderBy(df['col']) #create window 
lag_col = df.withColumn('new_col', lag('Col_to_lag_from', count=1).over(w))
df.show()

# Text Extraction 
greater_bool = df['col'].like('%Over Eight Years%')
less_bool = df['col'].like('%Less Eight Years%')

df = df.withColumn('new_col', (when(greater_bool, 1).when(less_bool, 0).otherwise(None)))

# Split, Pivot and Explode
split_col = split(df['col'], ',')
new_col = df.withColumn('someCol', split_col.getItem(0))

from pyspark.sql.functions import split, explode, lit, coalesce, first
df = df.withColumn('new_col', split(df['col'], ','))
ex_df = df.withColumn('ex_col', explode(df['new_col']))
ex_df = df.withColumn('constant_val', lit(1))
piv_df = df.groupBy('NO').pivot('ex_col').agg(coalesce(first('constant_val')))

# Binarizing, Bucketing and Encoding


# Spark Visualization

# Spark MLlib

In [ ]:
#Training and Testing Sets
data = sc.parallelize([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
training, test=data.randomSplit([0.6, 0.4])
training.collect()
test.collect()

## Collaborative Filtering
## The MSE is the average value of the square of (actual rating - predicted rating)
from pyspark.mllib.recommendation import ALS
r = Rating(user = 1, product = 2, rating = 5.0) #Rating class is a wrapper around tuple (user, product and rating)...(r[0], r[1], r[2])

model = ALS.train(ratings, rank, iterations) #Rank is the number of features
predictions = model.predictAll(unrated_RDD)
predictions.collect()

In [ ]:
## Classification

In [ ]:
PySpark MLlib contains specific data types Vectors and LabelledPoint.

Two types of Vectors
-Dense Vector: store all their entries in an array of floating point numbers
-Sparse Vector: store only the nonzero values and their indices.
    
denseVec = Vectors.dense([1.0, 2.0, 3.0])
DenseVector([1.0, 2.0, 3.0])

In [ ]:
A Labeledpoint is a wrapper for predicted value and input features.
positive = LabeledPoint(1.0, [1.0, 0.0, 3.0])
negative = LabeledPoint(0.0, [2.0, 1.0, 1.0])

In [ ]:
HashingTF() algorithm is used to map feature value to indices in the feature vector.
from pyspark.mllib.feature import HashingTF
sentence = "hello hello world"
words = sentence.split()
tf = HashingTF(10000) 
tf.transform(words)

In [ ]:
from pyspark.mllib.classification import LogisticRegressionWithLBFGS

data = [
        LabeledPoint(0.0, [0.0, 1.0]),
        LabeledPoint(1.0, [1.0, 0.0]),
]
RDD = sc.parallelize(data)
model = LogisticRegressionWithLBFGS.train(RDD)
lrm.predict([1.0, 0.0])
lrm.predict([0.0, 1.0])

In [ ]:
## Clustering

In [ ]:
from pyspark.mllib.clustering import KMeans

RDD = sc.textFile("WineData.csv"). \
       map(lambda x: x.split(",")).\
       map(lambda x: [float(x[0]), float(x[1])])
RDD.take(5)

model = KMeans.train(RDD, k = 2, maxIterations = 10)
model.clusterCenters

from math import sqrt
def error(point):
    center = model.centers[model.predict(point)]
    return sqrt(sum([x**2 for x in (point - center)]))
WSSSE = RDD.map(lambda point: error(point)).reduce(lambda x, y: x + y)
print("Within Set Sum of Squared Error = " + str(WSSSE))

wine_data_df = spark.createDataFrame(RDD, schema=["col1", "col2"])
wine_data_df_pandas = wine_data_df.toPandas()

cluster_centers_pandas = pd.DataFrame(model.clusterCenters, columns=["col1", "col2"])
cluster_centers_pandas.head()

plt.scatter(wine_data_df_pandas["col1"], wine_data_df_pandas["col2"]
plt.scatter(cluster_centers_pandas["col1"], cluster_centers_pandas["col2"], color="red", marker="x")